# Exercise 5

In [38]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

## 1. Crude Monte Carlo for ∫₀¹ eˣ dx
Estimate using n=100, report point estimate & 95% CI.

In [39]:
# Exercise 1: Crude MC
n = 100
u = np.random.rand(n)
fx = np.exp(u)
est1 = fx.mean()
se1 = fx.std(ddof=1) / np.sqrt(n)
ci1 = (est1 - 1.96*se1, est1 + 1.96*se1)
print(f"Estimate = {est1:.6f}")
print(f"95% CI  = [{ci1[0]:.6f}, {ci1[1]:.6f}]")

Estimate = 1.688072
95% CI  = [1.588390, 1.787753]


## 2. Antithetic Variables
Use u and 1−u, total 100 evaluations.

In [40]:
# Exercise 2: Antithetic
m = 50
u = np.random.rand(m)
fx2 = 0.5*(np.exp(u) + np.exp(1-u))
est2 = fx2.mean()
se2 = fx2.std(ddof=1) / np.sqrt(m)
ci2 = (est2 - 1.96*se2, est2 + 1.96*se2)
print(f"Antithetic estimate = {est2:.6f}")
print(f"95% CI            = [{ci2[0]:.6f}, {ci2[1]:.6f}]")

Antithetic estimate = 1.716744
95% CI            = [1.700109, 1.733379]


## 3. Control Variate
Use Y=u with E[Y]=0.5.

In [41]:
# Exercise 3: Control Variate
n = 100
u = np.random.rand(n)
h = np.exp(u)
c = np.cov(h, u, ddof=1)[0,1] / np.var(u, ddof=1)
cv = h - c*(u - 0.5)
est3 = cv.mean()
se3 = cv.std(ddof=1) / np.sqrt(n)
ci3 = (est3 - 1.96*se3, est3 + 1.96*se3)
print(f"Control variate est = {est3:.6f} (c={c:.3f})")
print(f"95% CI               = [{ci3[0]:.6f}, {ci3[1]:.6f}]")

Control variate est = 1.713477 (c=1.696)
95% CI               = [1.701401, 1.725552]


## 4. Stratified Sampling
Divide [0,1] into 10 strata, 10 samples each.

In [42]:
# Exercise 4: Stratified Sampling
k = 10
n_per = 10
ests = []
for i in range(k):
    a, b = i/k, (i+1)/k
    u = np.random.rand(n_per)*(b-a) + a
    ests.append(np.exp(u).mean())
est4 = np.mean(ests)
se4 = np.std(ests, ddof=1)/np.sqrt(k)
ci4 = (est4 - 1.96*se4, est4 + 1.96*se4)
print(f"Stratified est = {est4:.6f}")
print(f"95% CI         = [{ci4[0]:.6f}, {ci4[1]:.6f}]")

Stratified est = 1.710992
95% CI         = [1.396588, 2.025396]


## 5. Control Variate for Poisson Arrivals
Use control variates to reduce variance of blocking estimator.

In [43]:
def simulate_blocking_with_control_variate(servers=10, svc_mean=8, ia_mean=1, arrivals=10000):
    t_now = 0.0
    block_flags = []
    busy_history = []
    end_times = []

    for _ in range(arrivals):
        # generate next arrival time
        ia = np.random.exponential(scale=ia_mean)
        t_now += ia

        # remove finished services
        end_times = [et for et in end_times if et > t_now]

        busy = len(end_times)
        busy_history.append(busy)

        if busy < servers:
            # schedule service completion
            st = np.random.exponential(scale=svc_mean)
            end_times.append(t_now + st)
            block_flags.append(0)
        else:
            # arrival dropped
            block_flags.append(1)

    return np.array(block_flags), np.array(busy_history)


# perform multiple independent runs
runs = 10
all_blocks = []
all_busy = []

for _ in range(runs):
    blocks, busy = simulate_blocking_with_control_variate()
    all_blocks.append(blocks)
    all_busy.append(busy)

# stack results from all runs
blocks = np.hstack(all_blocks)
busy = np.hstack(all_busy)

# compute control variate parameter theta
cov_matrix = np.cov(blocks, busy, ddof=1)
cov_b_busy = cov_matrix[0, 1]
var_busy    = cov_matrix[1, 1]
theta       = cov_b_busy / var_busy
mean_busy   = busy.mean()

# adjust block indicators using the busy count control variate
blocks_adj = blocks - theta * (busy - mean_busy)

# estimate blocking probability and CI
block_rate_cv = blocks_adj.mean()
std_err      = blocks_adj.std(ddof=1) / np.sqrt(len(blocks_adj))
lower_bound  = block_rate_cv - 1.96 * std_err
upper_bound  = block_rate_cv + 1.96 * std_err

print(f"Estimated blocking fraction (with CV): {block_rate_cv:.4f}")
print(f"95% CI: ({lower_bound:.4f}, {upper_bound:.4f})")

Estimated blocking fraction (with CV): 0.1191
95% CI: (0.1174, 0.1208)


## 6. Common Random Numbers
Compare Poisson vs hyper-exponential arrivals using CRN.

In [44]:
def blocking_system_with_crn(rand_arrivals, rand_services, arrival_type="poisson", servers=10, avg_service=8, customers=10000):
    time_now = 0.0
    blocked_count = 0
    end_times = []
    avg_interarrival = 1.0

    for i in range(customers):
        u_arr = rand_arrivals[i]
        u_srv = rand_services[i]

        # generate interarrival
        if arrival_type == "poisson":
            delta = -np.log(u_arr) * avg_interarrival
        else:  # hyperexponential(0.8, rate1=0.8333; 0.2, rate2=5)
            if u_arr < 0.8:
                delta = -np.log(u_arr / 0.8) / 0.8333
            else:
                delta = -np.log((u_arr - 0.8) / 0.2) / 5.0

        # generate service time
        duration = -np.log(u_srv) * avg_service

        time_now += delta
        # remove completed services
        end_times = [t for t in end_times if t > time_now]

        if len(end_times) < servers:
            end_times.append(time_now + duration)
        else:
            blocked_count += 1

    return blocked_count / customers


# Run several paired simulations
trials = 10
n_cust = 10000
diffs = []

for _ in range(trials):
    uA = np.random.rand(n_cust)
    uS = np.random.rand(n_cust)

    p_block_pois = blocking_system_with_crn(uA, uS, "poisson", customers=n_cust)
    p_block_hyper = blocking_system_with_crn(uA, uS, "hyperexp", customers=n_cust)

    diffs.append(p_block_hyper - p_block_pois)

# compute mean difference and 95% CI
avg_diff = np.mean(diffs)
std_diff = np.std(diffs, ddof=1)
lo = avg_diff - 1.96 * std_diff / np.sqrt(trials)
hi = avg_diff + 1.96 * std_diff / np.sqrt(trials)

print(f"Mean block diff (hyperexp - poisson): {avg_diff:.4f}")
print(f"95% CI: ({lo:.4f}, {hi:.4f})")


Mean block diff (hyperexp - poisson): 0.0177
95% CI: (0.0166, 0.0189)


## 7. P(Z > a) via MC & Importance Sampling
Estimate for a=2,4 with n samples and IS N(a,σ²).

In [45]:
# Exercise 7: crude and IS
def estimate_prob(a, n=10000, sigma=1):
    z = np.random.randn(n)
    p_crude = np.mean(z > a)
    y = np.random.randn(n)*sigma + a
    w = stats.norm.pdf(y)/stats.norm.pdf(y, loc=a, scale=sigma)
    p_is = np.mean((y > a)*w)
    return p_crude, p_is
for a in [2,4]:
    pc, pi = estimate_prob(a)
    print(f"a={a}: crude={pc:.5f}, IS={pi:.5f}")

a=2: crude=0.02200, IS=0.02294
a=4: crude=0.00000, IS=0.00003


## 8. Exponential Importance Sampling
Use g(x)=lambdae^{-lambdax} for ∫₀¹ eˣdx, find optimal lambda.

In [46]:
# Exercise 8: Optimize lambda
def is_exp(n, lam):
    x = np.random.exponential(1/lam, size=n)
    w = (np.exp(x)*(x<=1)) / (lam*np.exp(-lam*x))
    return w.mean(), w.var(ddof=1)
lams = np.linspace(0.5,5,10)
results = [(lam, *is_exp(10000, lam)) for lam in lams]
for lam, m, v in results:
    print(f"lambda={lam:.2f}: est={m:.5f}, var={v:.5e}")

lambda=0.50: est=1.68943, var=5.90474e+00
lambda=1.00: est=1.74420, var=3.44343e+00
lambda=1.50: est=1.75814, var=3.27941e+00
lambda=2.00: est=1.70587, var=3.70753e+00
lambda=2.50: est=1.68618, var=4.89990e+00
lambda=3.00: est=1.70418, var=6.82489e+00
lambda=3.50: est=1.71947, var=9.70503e+00
lambda=4.00: est=1.79170, var=1.51108e+01
lambda=4.50: est=1.68284, var=2.01437e+01
lambda=5.00: est=1.69652, var=2.59646e+01


## 9. IS Estimator for Pareto Mean
Derive estimator using first-moment distribution and discuss.

**Discussion:**
- The IS estimator weights f(x)/g(x) where g∝x·f(x).
- This yields a constant weight, so variance is zero in theory.
- To cut down on variance, choose your sampling density so that you draw more points where the size of the integrand is large.
  Concretely, pick g(x) so that it is proportional to the absolute value of h(x) multiplied by the original density f(x).